# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,5.73,81,100,7.20,US,1701560324
1,1,alice springs,-23.7000,133.8833,29.75,26,2,7.72,AU,1701560325
2,2,antofagasta,-23.6500,-70.4000,18.60,80,22,3.56,CL,1701559864
3,3,waitangi,-43.9535,-176.5597,16.01,81,100,4.47,NZ,1701560325
4,4,tiksi,71.6872,128.8694,-28.15,99,100,2.98,RU,1701560326


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points("Lng","Lat", geo = True,tiles="OSM", color="City", size="Humidity",scale=0.8)

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[(city_data_df["Max Temp"]> 15) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] <20) ,:]

# Drop any rows with null values
final_vacation_df = vacation_df.dropna(how="any")

# Display sample data
final_vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,swan hill,-35.3378,143.5544,21.04,44,0,3.84,AU,1701560327
54,54,malvan,16.0667,73.4667,24.98,65,11,2.36,IN,1701560348
55,55,nkhata bay,-11.6066,34.2907,21.67,85,16,1.59,MW,1701560349
68,68,kingsville,27.5159,-97.8561,22.98,67,0,2.68,US,1701560164
76,76,wailua homesteads,22.0669,-159.3780,27.14,76,0,3.60,US,1701560360


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame().assign(City=final_vacation_df["City"], Country= final_vacation_df["Country"], Lat=final_vacation_df["Lat"],Lng=final_vacation_df["Lng"],Humidity=final_vacation_df["Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
353,al mishkhab,IQ,31.8042,44.4911,46,
286,kidal,ML,18.4411,1.4078,8,
277,chakari,ZW,-18.0629,29.8925,43,
352,ar rass,SA,25.8694,43.4973,44,
426,siyabuswa,ZA,-25.1132,29.0445,40,
291,sao vicente,BR,-23.9631,-46.3919,26,
392,nago,JP,26.5881,127.9761,60,
430,glen innes,AU,-29.7333,151.7333,31,
261,el sauzal de rodriguez,MX,31.9000,-116.6833,65,
198,orkney,ZA,-26.9802,26.6727,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [47]:
# Set parameters to search for a hotel
radius = 10000
geoapify_key = "855d2be2f0e943f6a18de905cd23cc74"
category = "accommodation.hotel"
params = {"categories": category,
   "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params).json()

    # Convert the API response to JSON format
#     name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
swan hill - nearest hotel: Murray Downs Golf Resort
malvan - nearest hotel: Hotel Silver Sand
nkhata bay - nearest hotel: Cairo Motel
kingsville - nearest hotel: Economy Inn
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
bardai - nearest hotel: No hotel found
wolmaransstad - nearest hotel: Buisfontein Safari Lodge
al kharijah - nearest hotel: Kharga Hotel
lihue - nearest hotel: Kauai Palms
tadine - nearest hotel: Hôtel Nengoné
bhadra - nearest hotel: No hotel found
al wajh - nearest hotel: فندق رالين 2
punta del este - nearest hotel: Hotel Aquarium
koungou - nearest hotel: Hôtel Trévani
kapa'a - nearest hotel: Pono Kai Resort
champerico - nearest hotel: Hotel y Restaurante El Submarino
sur - nearest hotel: Sur Hotel
mount gambier - nearest hotel: The Old Mount Gambier Gaol
orkney - nearest hotel: No hotel found
medina - nearest hotel: The Oberoi Madina
isangel - nearest hotel: Tanna Lodge
el sauzal de rodriguez - nearest hotel: Hotel Sausali

,City,Country,Lat,Lng,Humidity,Hotel Name
8,swan hill,AU,-35.3378,143.5544,44,Murray Downs Golf Resort
54,malvan,IN,16.0667,73.4667,65,Hotel Silver Sand
55,nkhata bay,MW,-11.6066,34.2907,85,Cairo Motel
68,kingsville,US,27.5159,-97.8561,67,Economy Inn
76,wailua homesteads,US,22.0669,-159.3780,76,Hilton Garden Inn Kauai Wailua Bay
86,bardai,NG,11.6500,7.2333,16,No hotel found
102,wolmaransstad,ZA,-27.1974,25.9831,45,Buisfontein Safari Lodge
107,al kharijah,EG,25.4514,30.5464,58,Kharga Hotel
109,lihue,US,21.9789,-159.3672,75,Kauai Palms
117,tadine,NC,-21.5500,167.8833,62,Hôtel Nengoné


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points("Lng","Lat", geo = True,tiles="ESRI",ylim=(-70, 60), color="City", hover_cols=["City","Country","Hotel Name","Humidity"])

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)